In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from datetime import datetime
from collections import defaultdict

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
# joint_df = pd.read_csv('./joint_clean_data/joint.csv', index_col=0)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# goal_domain = joint_df[['goal_domain']]

In [9]:
# goal_domain.to_csv('goal_domains.csv')

In [26]:
goal_domain = pd.read_csv('goal_domains.csv', index_col=0)

In [27]:
goal_domain.dropna(inplace=True)

In [28]:
goal_domain.head()

,goal_domain
0,Творчество и создание нового (Изобразительное ...
1,"Универсальные компетенции, гибкие навыки (soft..."
2,"Иностранные языки, гуманитарные знания и навык..."
3,"Прикладные знания и навыки, ручной труд (инфор..."
4,"Универсальные компетенции, гибкие навыки (soft..."


In [29]:
cv = CountVectorizer(max_df=0.5)
dtm = cv.fit_transform(goal_domain['goal_domain'])

In [30]:
LDA = LatentDirichletAllocation(n_components=11,
                               batch_size=256,
                               learning_decay=0.2) #playaround # 11 because we have 11 labels

In [31]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=256, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.2,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=11, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [32]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]]) #adjust
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['ручное', 'рукоделие', 'изобразительное', 'декор', 'музыка', 'изобретательство', 'литература', 'искусство', 'знания', 'гуманитарные', 'нового', 'творчество', 'создание', 'иностранные', 'языки']


THE TOP 15 WORDS FOR TOPIC #1
['экономика', 'педагогика', 'деятельность', 'управление', 'медицина', 'образование', 'психология', 'педагогических', 'методик', 'освоение', 'навыков', 'знания', 'гуманитарные', 'иностранные', 'языки']


THE TOP 15 WORDS FOR TOPIC #2
['садоводство', 'хозяйству', 'дому', 'транспортного', 'средства', 'вождение', 'работа', 'по', 'строительство', 'ручного', 'труда', 'знания', 'ручной', 'труд', 'прикладные']


THE TOP 15 WORDS FOR TOPIC #3
['профилактика', 'здоровья', 'улучшение', 'вид', 'внешний', 'правильное', 'веса', 'питание', 'контроль', 'полезные', 'привычки', 'самочувствие', 'спорт', 'внешность', 'зож']


THE TOP 15 WORDS FOR TOPIC #4
['графический', 'hard', 'самопрезентации', 'общения', 'командной', 'работы', 'тайм', 'самоорганизац

In [33]:
topics_tesaurus = {
    'label_attractor_knowledge':['знан', ],
    'label_attractor_hard_skill':[], 
    'label_attractor_soft_skill':['soft'],
    'label_attractor_tool':[],
    'label_attractor_community':['обществ', 'семья'],
    'label_attractor_subjectivity':[],
    'label_attractor_habits':[],
    'label_attractor_career':['карьер'],
    'label_attractor_fixing':[],
    'label_attractor_art':['худож', 'искусств', 'фотогр'],
    'label_attractor_health':['здоров'],
 
}

In [34]:
def add_cluster(x):
    group = np.nan
    stop = 0
    for key, values in topics_tesaurus.items():
        for val in values:
            if val in x:
                group = key
                stop = 1
                break
        if stop == 1:
            break
    if stop == 0:
        return group
    return group

In [35]:
goal_domain['goal_domain'] = goal_domain['goal_domain'].apply(lambda x: x.lower())

In [36]:
goal_domain['topic'] = goal_domain['goal_domain'].apply(lambda x: add_cluster(x))

In [37]:
goal_domain.head(20) #playaround head-tail, number

,goal_domain,topic
0,творчество и создание нового (изобразительное ...,label_attractor_art
1,"универсальные компетенции, гибкие навыки (soft...",label_attractor_knowledge
2,"иностранные языки, гуманитарные знания и навык...",label_attractor_knowledge
3,"прикладные знания и навыки, ручной труд (инфор...",label_attractor_knowledge
4,"универсальные компетенции, гибкие навыки (soft...",label_attractor_soft_skill
5,"универсальные компетенции, гибкие навыки (soft...",label_attractor_knowledge
6,"иностранные языки, гуманитарные знания и навык...",label_attractor_knowledge
7,"личные финансы, работа, бизнес и нормативное р...",NaN
8,"личные финансы, работа, бизнес и нормативное р...",label_attractor_knowledge
9,творчество и создание нового (фотография)униве...,label_attractor_soft_skill


In [38]:
goal_domain[goal_domain['topic'].isna() == True].shape

(8117, 2)

In [40]:
goal_domain[goal_domain['topic'].isna() == True].head(50)

,goal_domain,topic
7,"личные финансы, работа, бизнес и нормативное р...",NaN
10,"математика и it (работа с данными, программиро...",NaN
11,"личные финансы, работа, бизнес и нормативное р...",NaN
12,математика и it (работа с данными)математика и...,NaN
13,"личные финансы, работа, бизнес и нормативное р...",NaN
14,юриспруденция,NaN
16,"личные финансы, работа, бизнес и нормативное р...",NaN
17,психология и педагогика,NaN
20,"математика и it (работа с данными, прочие цифр...",NaN
22,психология творчество и создание нового (новые...,NaN


In [ ]:
import pickle

In [ ]:
with open('topics_tesaurus.pickle', 'wb') as f:
    pickle.dump(topics_tesaurus, f)